# Init

In [ ]:
import numpy as np
import pandas as pd
import logging
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
from pathlib import Path
import json
%cd ..
%load_ext autoreload

In [ ]:
%autoreload 2
from src.MapData import MapData, min_max_dist, triangle_corr, get_triangular_kernel
from src.LineFinder import LineFinder

from src.plotting_functions import plot_embedding, plot_single_variable_map, _update_layout

# Data

In [ ]:
map_data = MapData(
    './data/2022_03_22_P56B_307x532.libsdata',
    overwrite=False
)
map_data.get_metadata()
map_data.load_wavelenths()
map_data.load_all_data('preprocessed')
# map_data.load_all_data('trial')
# map_data.spectra = map_data.spectra[:200,:]
map_data.get_map_dimensions()

map_data.trim_spectra(64)
map_data.estimate_systemic_noise()

map_data.get_baseline(
    min_window_size=50,
    smooth_window_size=100
)
map_data.baseline_correct()

map_data.upsample_spectra()
if map_data.systemic_noise_spectrum is not None:
    map_data.denoise_spectra(
        file_name_supplement='preprocessed',
        threshold=np.std(map_data.systemic_noise_spectrum)
    )

maxima_spectrum = map_data.spectra.max(axis=0)

## Line finding

In [ ]:
# for bin in bins:
#     plt.scatter(
#         x=bin,
#         y=np.std(maxima_spectrum[maxima_spectrum <= bin + bin_half_width])
#     )

In [ ]:
line_finder = LineFinder(
    maxima_spectrum,
    map_data.wvl,
    name='maxima'
)
line_finder.find_lines(
    # height=bins[2],
    # threshold=None,
    # distance=None,
    # prominence=bins[1],
    # width=7,
    # wlen=35,
    # rel_height=1,
)
line_finder.load_nist_tables(
    # Path('D:/OneDrive - Vysoké učení technické v Brně/projects/marsData/inventory/nistTables')
    Path('C:/Users/kepes/OneDrive - Vysoké učení technické v Brně/projects/marsData/inventory/nistTables')
)
line_finder.find_peaks_in_reference(
    maxima_spectrum, 
    scale=False,
    show_cond=False
)

line_finder.plot.show()

## Line intensities

In [ ]:
map_data.set_emission_line_parameters(
    line_finder.peaks[1].get('left_bases'),
    line_finder.peaks[1].get('right_bases'),
    line_centers=line_finder.peaks[0]
)
map_data.set_emisssion_line_functions(    
    intensity_funcs=[np.max,np.sum,min_max_dist,triangle_corr]
)
map_data.get_emission_line_intensities()

## Line ratios

In [ ]:
line_ratios = dict()
for func in map_data.line_intensities:
    line_ratios[func] = dict()
    for ref_line in map_data.line_intensities[func]:
        line_ratios[func][f'div_by_{ref_line}'] = {
            f'{line}/{ref_line}':np.divide(
                intensities,
                map_data.line_intensities.get(func).get(ref_line),
            )
            for line,intensities
            in map_data.line_intensities.get(func).items()
            if ref_line != line
        }

In [ ]:
for normalization_id in line_ratios:
    for line,intensities in line_ratios[normalization_id].items():
        file_name = "P56B/{}/line_max_{}_nm".format(
            normalization_id.replace('.','_'),
            line.replace('.','_').replace('/','by')
        )
        print(file_name)
        if not Path('./temp').joinpath(file_name).parent.exists():
            Path('./temp').joinpath(file_name).parent.mkdir(parents=True)
        plot_single_variable_map(
            plot_values=map_data.vector_to_array(intensities),
            file_id=file_name,
            figure_title=f'{line} nm',
            colorbar_title='Intensity',
            fig_size_scaling=3.,
            cutoff_quantile=.99
        )

        plt.close()

## Saving maps

In [ ]:
for intensity_func in map_data.line_intensities:
    for line,intensities in map_data.line_intensities[intensity_func].items():
        file_name = f"P56B/denoised/intensityMaps/{intensity_func}_{line.replace('.','_')}_nm"
        if not Path('./temp').joinpath(file_name).parent.exists():
            Path('./temp').joinpath(file_name).parent.mkdir(parents=True)
        print(file_name)
        plot_single_variable_map(
            plot_values=map_data.vector_to_array(intensities),
            file_id=file_name,
            figure_title=f'{line} nm',
            colorbar_title='Intensity',
            fig_size_scaling=3.,
            cutoff_quantile=.99
        )

        plt.close()

# Embeddings

In [ ]:
from sklearn.decomposition import PCA
from umap import UMAP
from sklearn.manifold import TSNE

In [ ]:
embeddings = dict()
embedders = dict()
embedders['pca'] = PCA(n_components=3)
# embedders['tsne'] = TSNE(
#     n_components=3,
#     perplexity=30,
#     learning_rate=200,
#     metric='cosine'
# )
# embedders['umap'] = UMAP(
#     n_components=3, 
#     n_neighbors=30, 
#     min_dist=0.5
# )

In [ ]:
for embedding_method in embedders:
    try:
        # embeddings[embedding_method] = np.load(f'./temp/embeddings_{embedding_method}.npy')
        pass
    except FileNotFoundError:
        print(f'embedding using {embedding_method} not found, creating...')
        embeddings[embedding_method] = embedders[embedding_method].fit_transform(
            pd.DataFrame(map_data.line_intensities)
        )

In [ ]:
pd.DataFrame(
    map_data.line_intensities.get('sum')
).drop(columns=['266.08'])

In [ ]:
embedding_method = 'pca'

embeddings[embedding_method] = embedders[embedding_method].fit_transform(
    pd.DataFrame(
        map_data.line_intensities.get('sum')
    ).drop(columns=['266.08'])
)

fig = plot_embedding(
    embeddings[embedding_method],
    # explained_variances=pca_model.explained_variance_ratio_.copy(),
    # colors=predicted_labels[clustering_method],
    marker_size=2,
    return_figure=True
)
# fig.suptitle(clustering_method)
fig.patch.set_alpha(0)
fig.tight_layout()
# fig.savefig(
#     f'./temp/{embedding_method}_embeddings_full_spectra_clustering_using_{clustering_method}_{segment}',
#     transparent=True
# )

# from joblib import load
# pca_model = load('../temp/pca_model.joblib')

In [ ]:
plot_data = map_data.vector_to_array(embeddings['pca'][:,1])
plot_single_variable_map(
    plot_data,
    fig_size_scaling=2
)

# Edge detection

In [ ]:
import cv2

In [ ]:
# map_data._line_intensities_to_arrays()

In [ ]:
# edge_data = map_data.vector_to_array(embeddings['pca'][:,1])
# edge_data = map_data.vector_to_array(line_ratios['sum']['div_by_279.63']['393.34/279.63'])
edge_data = map_data.vector_to_array(map_data.line_intensities['sum']['393.34'])
plot_single_variable_map(
    edge_data,
    fig_size_scaling=2
)


In [ ]:
# edge_data = map_data.vector_to_array(embeddings['pca'][:,1])
edge_data = map_data.vector_to_array(map_data.line_intensities['sum']['393.34'])
edge_data -= edge_data.min()
edge_data /= edge_data.max()
edge_data *= 255
edge_data = edge_data.astype(np.uint8)

th, im_th = cv2.threshold(
    edge_data, 
    10,
    230,
    cv2.THRESH_BINARY
)
plt.imshow(im_th)

In [ ]:
# Copy the thresholded image.
im_floodfill = im_th.copy()

im_floodfill = cv2.filter2D(
    im_floodfill, 
    -1, 
    np.ones(shape=(7,7))/49
    # np.ones(shape=(20,20))/400
)
  
# Mask used to flood filling.
# Notice the size needs to be 2 pixels than the image.
h, w = im_th.shape[:2]
mask = np.zeros((h+2, w+2), np.uint8)
  
# # Floodfill from point (0, 0)
cv2.floodFill(im_floodfill, mask, (0,0), 255)
plt.imshow(im_floodfill)

In [ ]:
plt.imshow(im_floodfill != 255)

In [ ]:
(im_floodfill != 255).reshape(-1,1)

In [ ]:
kept_spectra_ratio = np.sum(im_floodfill != 255)\
  /(im_floodfill.shape[0] * im_floodfill.shape[1])
print(f'{kept_spectra_ratio:.2f}')

In [ ]:
cluster_data = map_data.spectra[(im_floodfill != 255).reshape(-1),:]

In [ ]:
reduced_embeddings = embedders['pca'].fit_transform(
    cluster_data
)

In [ ]:
fig = plot_embedding(
    reduced_embeddings,
    marker_size=2,
    return_figure=True
)
fig.patch.set_alpha(0)
fig.tight_layout()

In [ ]:
plot_data = np.zeros(map_data.spectra.shape[0])
plot_data[(im_floodfill != 255).reshape(-1)] = reduced_embeddings[:,0]

In [ ]:
plot_single_variable_map(
    map_data.vector_to_array(plot_data),
    fig_size_scaling=3
)

In [ ]:
embedding_method = 'umap'
image_data = np.concatenate([
    map_data.vector_to_array(
        embeddings[embedding_method][:,0]
    )[...,np.newaxis],
    map_data.vector_to_array(
        embeddings[embedding_method][:,1]
    )[...,np.newaxis],
    map_data.vector_to_array(
        embeddings[embedding_method][:,2]
    )[...,np.newaxis]],
    axis=2
)

image_data -= image_data.min()
image_data /= image_data.max()

plt.figure(figsize=(9,9))
plt.imshow(
    image_data
)

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        y=maxima_spectrum
    )
)

# Autocorrelation

In [ ]:
import statsmodels.api as sm

x = maxima_spectrum[3500:3600].copy()
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        y=x
    )
)

# Plot the correlogram
sm.graphics.tsa.plot_acf(x, lags=99)
plt.show()

In [ ]:
pd.plotting.autocorrelation_plot(x)

In [ ]:
def get_autocorrelation(x):
    x = x.copy()
    x -= np.mean(x)
    autocorrelation = np.correlate(x,x,mode='full')
    autocorrelation /= np.var(x)
    return autocorrelation[len(autocorrelation)//2:]

In [ ]:
data = np.apply_along_axis(
    func1d=get_autocorrelation,
    axis=1,
    arr=map_data.spectra[:,3500:3600]
)

In [ ]:
base_size = 48

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=map_data.wvl[3500:3600],
        y=map_data.spectra[51521,3500:3600]
    )
)
fig = _update_layout(fig)

fig.update_layout(
    title_font_size=base_size,
    xaxis_title_font_size=base_size-12,
    yaxis_title_font_size=base_size-12,
    xaxis_tickfont_size=base_size-18,
    yaxis_tickfont_size=base_size-18,
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)'
)

pio.write_image(
    fig, 
    './output/molecularBand.png',
    height=1200,
    width=1800
)

In [ ]:
def get_extremum_locations(x):
    maxima = np.where(np.diff(np.sign(np.diff(x))) == -2)[0] + 1
    minima = np.where(np.diff(np.sign(np.diff(x))) == 2)[0] + 1
    return (maxima, minima)

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        y=get_autocorrelation(map_data.spectra[0,3500:3600])
        # y=data[0]
    )
)
_update_layout(fig,x='Wavelength shift',y='Correlation coefficient')
fig.update_layout(
    title_font_size=base_size,
    xaxis_title_font_size=base_size-12,
    yaxis_title_font_size=base_size-12,
    xaxis_tickfont_size=base_size-18,
    yaxis_tickfont_size=base_size-18,
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)'
)

pio.write_image(
    fig, 
    './output/autocorrelationFunction.png',
    height=1200,
    width=1800
)

In [ ]:
extremum_locations = [
    get_extremum_locations(x)
    for x
    in data
]

In [ ]:
plot_data = np.zeros(len(extremum_locations))
for ndx in range(len(plot_data)):
    try:
        plot_data[ndx] = extremum_locations[ndx][0][1] - extremum_locations[ndx][1][1]
    except IndexError:
        plot_data[ndx] = 0

plot_single_variable_map(
    map_data.vector_to_array(plot_data),
    fig_size_scaling=2,
    file_id='autocorrelationDistances'
)

In [ ]:
plot_single_variable_map(
    map_data.vector_to_array(np.abs(data).sum(axis=1)),
    fig_size_scaling=2,
    file_id='autocorrelationArea'
)

In [ ]:
print(np.where(np.diff(np.sign(np.diff(autocorrelation))) == -2)[0] + 1)
print(np.where(np.diff(np.sign(np.diff(autocorrelation))) == 2)[0] + 1)

In [ ]:
np.diff(np.where(np.diff(np.sign(np.diff(autocorrelation))) == 2)[0])

In [ ]:
autocorrelation[np.where(np.diff(np.sign(np.diff(autocorrelation))) == -2)[0] + 1]
autocorrelation[np.where(np.diff(np.sign(np.diff(autocorrelation))) == 2)[0] + 1]